# TME 3.2: Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm random Projections.


1\.  Nous allons réutiliser les fonctions du precedent pour générer t séquences artificielles de taille n, et implanter dans chaque séquence un motif de taille k à des positions aléatoires avec v substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [2]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence
f= 0.5 #frequence de variation du motif.


In [3]:
from math import floor

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront en majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    
    for i in range (n):
        tmp=random.choice(nuc)
        sequence =sequence+tmp
        
    if upper== False:
        sequence= sequence.lower()
    return sequence


def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    tmp=''
    randind=0
    up=upper
    
    listIndDéjaMODIF=[]
    
    for pos in range(nbpos): #On modifie autant de position possible que la taille du motif le permet 
        if pos < len(motifM)-1:
            #Tirer un caractere aléatoire tmp parmi atgc et l'insérer à la posiion randind choisie aléatoirement 
            tmp = generateRandomSequence(1, up)
            randind= random.randint(0, len(motif)-1)
            
            #Pour ne pas modifier à 2 reprises à un meme indice
            if randind in listIndDéjaMODIF: #On doit tirer un nouvel indice aléatoirevement
                
                indConnue = True
                while indConnue:
                    randind= random.randint(0, len(motif)-1)
                    if randind not in listIndDéjaMODIF:
                        indConnue = False
            
            #Modification à la position choisit aléatoirement
            motifM[randind]=tmp
            listIndDéjaMODIF.append(randind)
        
    return "".join(motifM)

#tester modifMotif
print (modifierMotif("acg", 2))


def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    sequences = []
    motifRand=generateRandomSequence(k,False)
    print('Motif à implanté dans 50% des cas: ',motifRand)
    motifTMP=''
    seqTPM=''
    tailleSeqSanMotifImplanté = n-k
    positionRand=random.randint(0,tailleSeqSanMotifImplanté)
    
    #Détermine nb de séquence pour lesquelles le motif implanté ne doit pas varier, dans f des cas le motif ne doit varier
    SeqAvecVarDeMotif= floor(f*t) #Partie entiére inférieure 
    Compteur=0
    
    for sep in range(t): #Création des séquences avec le motif générer qui varie de v base et pour une fréquence f
        seqTPM=generateRandomSequence(tailleSeqSanMotifImplanté,True) #Sequence en majuscule
        
        #Choix du motif
        if Compteur == SeqAvecVarDeMotif: 
            #Stop l'insertion de Motif avec variation
            motifTMP=motifRand
        else: 
            motifTMP= modifierMotif (motifRand,v,False)        #Motif en minuscule
            Compteur = Compteur+1
        
        #Insertion du motif dans la seq et ajout à la liste résultante cad la matrice
        seqTPM= insertMotif(seqTPM, motifTMP, positionRand)
        sequences.append(seqTPM) #list(seqTPM)
    
        #Initialisation d'une position pour insérer motif de la prochaine séquence 
        positionRand=random.randint(0,tailleSeqSanMotifImplanté) 
        
    return sequences

adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

CTg
Motif à implanté dans 50% des cas:  ggact
['TggacgTCGG', 'CCCggactGG', 'CATggactGT']
['TGGACGTCGG', 'CCCGGACTGG', 'CATGGACTGT']


2\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de p à k, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [4]:
def getRandomFixePositions(p, k):
    """
    Genere une projection de p vers k
    entrée p: nombre de positions choisi pour la projection 
    entrée k: nombre de positions du motif original
    sortie projection: liste de positions choisi aléatoirement
    """
    projection = []
    #count = 0
    
    for e in range(p):
        p= random.randint(0, k-1)
        #Pour ne pas avoir à 2 reprises un meme indice
        if p in projection: #On doit tirer un nouvel indice aléatoirevement
            projConnue = True
            while projConnue:
                p= random.randint(0, k-1)
                if p not in projection:
                    projConnue = False
        
        #Ajout de la projection à la liste
        projection.append(p)
    
    return projection

lR = getRandomFixePositions(4, 7)
lR.sort()
print (lR)

def generateKey(projection, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée projection : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    cle = ""
    
    for i in (projection):
        if i< len(motif):
            cle= cle + motif[i]
    
    return cle

key=generateKey(lR, "01234567")
print(key)

[2, 4, 5, 6]
2456


3\. Implémenter l'algorithme ``randomProjection``. Bonnus : Pour ameliorer la performance vous pouvez abandonner les motifs de taille k peu complexes.

In [42]:
def randomProjection(k, p, sequences):
    """
    Implémente l'algorithme randomProjection
    entrée k : taille du motif
    entrée p : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur= original motif
    """
    motifs  = {}; 
    motifsSeq  = {}
    
    p=getRandomFixePositions(p, k)
    p.sort()
    #print(p)
    
    for s in sequences:
        if len(s)>=k:
            for i in range(len(s)-k+1):
                motif = s[i:i+k]
                key=generateKey(p, motif)
                
                if key in motifs.keys() and key in motifsSeq.keys():
                    #Si la clé existe déja
                    motifs[key]+=1
                    motifsSeq[key]=motifsSeq[key]+[motif]
                
                else:
                    #Si la clé n'existe pas
                    motifs[key]=1
                    motifsSeq[key]=[motif]
    
    return motifs, motifsSeq

#motifsSort = sorted(motifs, reverse=True, key=motifs.get)

adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
motifsProj, motifsSeq = randomProjection(7, 4, adnTest)
print (motifsProj)
print (motifsSeq)

#motifsProj => {'TACG': 1, 'TCGC': 3, 'AGGA': 1, 'AGCC': 1, 'GCAG': 1, 'CACC': 1, 'CGCA': 1, 'AGGC': 1, 'CGCG': 1, 'CCCT': 1}
#motifsSeq => {'TACG': ['TTAACGG'], 'TCGC': ['TAACGGC', 'TCACGCC', 'TACCGGC'], 'AGGA': ['AACGGCA'], 'AGCC': ['ACGGCAC'], 'GCAG': ['GCTCACG'], 'CACC': ['CTCACGC'], 'CGCA': ['CACGCCA'], 'AGGC': ['ACCGGCC'], 'CGCG': ['CCGGCCG'], 'CCCT': ['CGGCCGT']}

{'TCGG': 1, 'AGGC': 2, 'AGCA': 1, 'CCAC': 1, 'CACG': 1, 'TCGC': 1, 'CGCC': 2, 'ACCA': 1, 'CCCG': 1, 'GCGT': 1}
{'TCGG': ['TTAACGG'], 'AGGC': ['TAACGGC', 'TACCGGC'], 'AGCA': ['AACGGCA'], 'CCAC': ['ACGGCAC'], 'CACG': ['GCTCACG'], 'TCGC': ['CTCACGC'], 'CGCC': ['TCACGCC', 'ACCGGCC'], 'ACCA': ['CACGCCA'], 'CCCG': ['CCGGCCG'], 'GCGT': ['CGGCCGT']}


4\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

reponse: Le motif implanté admet TCGC comme clé fixe associé à la projection car on constate que c'est la bucket la plus grande, dans votre exemple 3 motifs ont été trouvés... avec des variations du motif pour les positions non projetées.

Puisque les projections sont aléatoires, nous parvenons à obtenir la clé TCGC mais pas avec les bonnes projections aléatoires pour retrouver le motif avec des variations.
L'algorithme randomProjection demande donc d'être exécuter un grand nombre de fois avant de nous permettre d'obtenir un résultat satisfaisant.

D'autres projections nous permettent tout de même de tirer des conclusions, par exemple la bucket la plus grande avec la projection [0, 2, 3, 6] permet de constater une unique clé qui a la plus grande fréquence càd 'TACC': 2,
et 'TACC': ['TAACGGC', 'TCACGCC']. Ce qui nous donne une idée de l'allure du motif éventuellement, d'autant plus que ce phénomène se reproduit pour de nombreuse projection ou on a une unique liste (taille 2) plus longue que les autres qui ont tous une taille de 1.



5\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [43]:
#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    #nuc = ('A', 'C', 'G', 'T')
    q = len(nuc)
    PWM = np.zeros((q, k))
    
    for mot in motifs :
        for i in range (k): #i=indice colonne
            indLigne = nuc.index(mot[i]) #indiceLigne
            PWM[indLigne,i]=PWM[indLigne,i]+1
    return PWM

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0
    nbLigne = len(MF)
    nbColonne = len(MF[0])
    
    for c in range(nbColonne):
        MAX=0
        for l in range(nbLigne):
            #print(MF[l,c])
            if MF[l,c]>MAX:
                MAX=MF[l,c]
        #print("Sortie")
        sc= sc + MAX
        #print("Valeur sc =", sc)

    return sc

In [49]:
def randomProjIt(sequences, k, v, nuc, It):
    """
    Implémente l'algorithme randomProjection version iteractive
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée v : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
    #NOUS ENVISAGEONS D'OPTIMISER LA FONCTION POUR IGNORER LES MOTIFS PEU COMPLEX, APRÉS LE DÉPOT
    
    motifss = []; scores = 0
    
    for i in range(It):
        (motifs, motifsSeq)=randomProjection(k, v, sequences)
        keys= motifsSeq.keys()
        
        for ke in keys:
            MF=profile(motifsSeq[ke], k, nuc)
            scoretmp=getScore(MF, k)
            
            if scoretmp> scores:
                scores= scoretmp
                motifss= motifsSeq[ke]  
                
    return scores, motifss

score, seqsMotif = randomProjIt(adnTest, 7, 4, nuc, 100)
print (score, seqsMotif)

#Nous retrouvons la bucket la plus longue de notre question precedente et donc le motif avec pour clé TCGC càd les positions 0,3,4 et 6^^


18.0 ['TAACGGC', 'TCACGCC', 'TACCGGC']


6\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [57]:
def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

def complement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in seq])

def printTopFMotifsFreq(motifs, m, rev=False):
    motifsRet = {}
    motifsSort = sorted(motifs, reverse=True, key=motifs.get)
    i = 0
    while (i < m):
        motifPrint = motifsSort[i]
        print (motifsSort[i])
        if rev:
            motifPrint = reverseComplement(motifsSort[i])
        print (i, motifPrint, "-", motifs[motifsSort[i]])
        i = i + 1
        
printTopFMotifsFreq(motifsProj, 10)
print (motifsSeq['TCGC'])

AGGC
0 AGGC - 2
CGCC
1 CGCC - 2
TCGG
2 TCGG - 1
AGCA
3 AGCA - 1
CCAC
4 CCAC - 1
CACG
5 CACG - 1
TCGC
6 TCGC - 1
ACCA
7 ACCA - 1
CCCG
8 CCCG - 1
GCGT
9 GCGT - 1
['CTCACGC']


In [62]:
k=7; p=4; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_1.fasta"

sequencesChip   = readFasta(genome, n)
t = len(sequencesChip)
print (sequencesChip[8], t, n, k)
revSequences = [reverseComplement(m) for m in sequencesChip]

sequences = sequencesChip + revSequences

score, seqsMotif = randomProjIt(sequences, k, p, nuc, 100)

print(score, seqsMotif)

TCTAATCCTTTAGGTATTACAGGTAATATGGATAGAATTGGAATGCATGGTTATTTCATTTTTAAAGATTTAATTACTGT 367 80 7
4108.0 ['TATTTTT', 'TTTTATT', 'TATTGTT', 'TATTTGT', 'TTTTCAT', 'TCTTATT', 'TATTATT', 'TATTATT', 'TGTTATT', 'TATTATT', 'TTTTAAT', 'TATTCCT', 'TATTATT', 'TATTGGT', 'TCTTAGT', 'TATTGCT', 'TATTCCT', 'TATTATT', 'TATTGCT', 'TATTCTT', 'TCTTAAT', 'TTTTCTT', 'TATTTGT', 'TATTCTT', 'TATTCCT', 'TATTTAT', 'TATTCCT', 'TGTTATT', 'TATTATT', 'TTTTATT', 'TATTTAT', 'TATTACT', 'TATTATT', 'TATTTTT', 'TTTTTGT', 'TATTAGT', 'TGTTTAT', 'TGTTAAT', 'TATTTGT', 'TGTTATT', 'TTTTATT', 'TATTAAT', 'TGTTATT', 'TATTCCT', 'TGTTGAT', 'TGTTATT', 'TATTCAT', 'TTTTGCT', 'TATTCCT', 'TGTTATT', 'TATTGGT', 'TATTAGT', 'TTTTGAT', 'TATTAAT', 'TTTTTAT', 'TTTTATT', 'TATTATT', 'TATTTGT', 'TATTAAT', 'TATTAGT', 'TTTTAAT', 'TTTTCTT', 'TCTTCTT', 'TCTTAGT', 'TATTGAT', 'TATTTTT', 'TTTTTAT', 'TATTCAT', 'TTTTCAT', 'TCTTTAT', 'TATTTGT', 'TATTGCT', 'TGTTATT', 'TATTATT', 'TATTTTT', 'TTTTTGT', 'TATTGGT', 'TGTTATT', 'TATTATT', 'TATTGCT', 'TCTTATT', 'TGTTCAT', 